<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

Fuente: [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.07-Merge-and-Join.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


In [4]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

# Combinando Datasets: Merge y Join

In [2]:
import pandas as pd
import numpy as np

## Categorias de Joins

La función ``pd.merge()`` implementa varios tipos de `joins`:
- Uno-a-uno (*one-to-one*)
- Muchos-a-uno (*many-to-one*)
- Muchos-a-muchos (*many-to-many*).

Los 3 tipos de `join` se acceden desde ``pd.merge()``. El tipo de `join` dependerá de los datos.



### One-to-one

Es la forma mas simple de `join`.

In [5]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en sólo ``DataFrame``, utilizamos la función ``pd.merge()``:

In [6]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función ``pd.merge()`` reconoce que cad `DataFrame`contiene una columna `employee` y automaticamente las une con esta columna como `key`.

El resultado de esta unión es un nuevo ``DataFrame`` que combina la información de las dos tables.

Observar que el order de las entradas en cada columna no se conserva: en este caso, el orden de la columna `emploee` difiere entre``df1`` y ``df2``, y la función ``pd.merge()`` lo tiene en cuenta.

Además, el merge, en general, descarta los indices, salvo casos especiales.

### Many-to-one

Many-to-one son `join` en los cuales one de las dos columnas con key contiene entradas duplicadas.
El ``DataFrame`` resultante conserva los registros duplicados.


In [8]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                        'supervisor': ['Carly', 'Guido', 'Steve']})
df_many_to_one = pd.merge(df3, df4)
df_many_to_one

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


In [9]:
display('df3', 'df4', 'df_many_to_one')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

df_many_to_one
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

El ``DataFrame`` resultante tiene una columna con una columna "supervisor", donde la información se repite en una o mas ubicaciones según corresponda.

### Many-to-many

Los `join` Many-to-many son mas confusos en términos conceptuales, pero se pueden definir con claridad.

Si la columna de la _key_ tanto del conjunto de la izquierda como en el de la derecha contiene repeticiones, el resultado es un many-to-many.

Con un ejemplo puede quedar mas claro. Consideremos un ``DataFrame`` mostrando uno o mas skills vinculados a un grupo en particular.

Realizando una `join` many-to-many, podemos recuperar los skills asociados a cualquier individuo:n:

In [10]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
many_to_many = pd.merge(df1, df5)
many_to_many

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


In [11]:
display('df1', 'df5', 'many_to_many')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


En la práctica, nunca vamos a tener un dataset tan limpio y preciso como los de ejemplo.

En las siguientes secciones vamos a considerar algunas de las opciones que provee ``pd.merge()`` que permiten perfeccionar las operaciones de `join`.

## Especificaciones de la _key_ para el merge

En la situación por defecto,  ``pd.merge()`` busca uno o mas nombres de columnas que _matcheen_  entre las dos tablas, y las utiliza como las _keys_.

Sin embargo, los nombres de las columnas no siempre _matchean_ tan bien, y por esto es que ``pd.merge()`` brinda variedad de opciones para solucionar esto.

### keyword ``on`` 

Uno puede explicitamente especificar la columna _key_ utilizando ``on`` con el nombre o la lista de columnas para utilizar


In [13]:
df_on = pd.merge(df1, df2, on='employee')

In [14]:
display('df1', 'df2', 'df_on')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

df_on
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opciones sólo funciona si ambos ``DataFrame`` tienen la columna especificada.

### keywords ``left_on`` y ``right_on``

En situaciones es posible querer realizar un _merge_ sobre dos datasets que tienen diferentes nombres de columnas.

En este caso, es posible utilizar ``left_on`` y ``right_on`` para especificar los nombres de ambas columnas.

In [18]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
df_left_right = pd.merge(df1, df3, left_on="employee", right_on="name")
df_left_right

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


In [17]:
display('df1', 'df3', 'df_left_right')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

df_left_right
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

El resultado tiene una columna adicional que podemos eliminar mediante ``drop()``

In [19]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000
